In [1]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
medicines = pd.read_csv('/kaggle/input/medicine/medicine.csv')

In [3]:
medicines.head()

In [4]:
medicines.shape

In [5]:
medicines.isnull().sum()

In [6]:
medicines.dropna(inplace=True)

In [7]:
medicines.duplicated().sum()

In [8]:
medicines['Description']

In [9]:
medicines['Description'].apply(lambda x:x.split())

In [10]:
medicines['Reason'] = medicines['Reason'].apply(lambda x:x.split())
medicines['Description'] = medicines['Description'].apply(lambda x:x.split())

In [11]:
medicines['Description'] = medicines['Description'].apply(lambda x:[i.replace(" ","") for i in x])

In [12]:
medicines['Description'] = medicines['Description'].apply(lambda x:[i.replace(" ","") for i in x])

In [13]:
medicines['tags'] = medicines['Description'] + medicines['Reason'] 

In [14]:
new_df = medicines[['index','Drug_Name','tags']]

In [15]:
new_df

In [16]:
new_df['tags'].apply(lambda x:" ".join(x))

In [17]:
new_df

In [18]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [19]:
new_df

In [20]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [21]:
new_df

In [22]:
!pip install nltk

In [23]:
import nltk

In [24]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [25]:
!pip install -U scikit-learn scipy matplotlib

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english',max_features=5000)

In [28]:
def stem(text):
  y = []

  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y)  

In [29]:
new_df['tags'] = new_df['tags'].apply(stem)

In [30]:
cv.fit_transform(new_df['tags']).toarray().shape

In [31]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [32]:
cv.get_feature_names()

In [33]:
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
cosine_similarity(vectors)

In [35]:
similarity = cosine_similarity(vectors)

In [36]:
similarity[1]

In [37]:
def recommend(medicine):
    medicine_index = new_df[new_df['Drug_Name'] == medicine].index[0]
    distances = similarity[medicine_index]
    medicines_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in medicines_list:
        print(new_df.iloc[i[0]].Drug_Name)
    

In [38]:
recommend("Paracetamol 125mg Syrup 60mlParacetamol 500mg Tablet 10'S")

In [39]:
import pickle

In [40]:
pickle.dump(new_df.to_dict(),open('medicine_dict.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))